In [90]:
import requests
import json
import time

url = 'https://api.ahighapi.com/Market/marketList?symbol=wdt_usdt'
headers = {'User-Agent':'Mozilla/5.0 \(Windows NT 6.1; WOW64\) AppleWebKit/537.36 \(KHTML, like Gecko\) Chrome/39.0.2171.71 Safari/537.36'}
response = requests.get(url, headers = headers)

orderFormDict = json.loads(response.text)
bids = orderFormDict['data']['depth']['bids']
asks = orderFormDict['data']['depth']['asks']
print('success')

success


In [91]:
import pandas as pd
import numpy as np
from pandas import DataFrame 


directory = '/data/coins/'
currentDateWithTime = time.strftime('%Y-%m-%d %H点%M分',time.localtime(time.time()))
fileName = '[WdtCoin]' + str(currentDateWithTime) + '.xlsx'

bids_df = pd.DataFrame(bids, columns=['委托价', '数量', '总额']) 
asks_df = pd.DataFrame(asks, columns=['委托价', '数量', '总额']) 
bids_df['委托价'] = bids_df['委托价'].apply(float)
bids_df['数量'] = bids_df['数量'].apply(float)
bids_df['总额'] = bids_df['总额'].apply(float)
asks_df['委托价'] = asks_df['委托价'].apply(float)
asks_df['数量'] = asks_df['数量'].apply(float)
asks_df['总额'] = asks_df['总额'].apply(float)


def getBriefOrderList(df):
    result = DataFrame([], columns=['委托价', '数量', '总额'])
    integerList = []
    for index, row in df.iterrows():
        figure = int(row.委托价) 
        if figure not in integerList:
            integerList.append(figure)
    StatisticalMap = {}
    for i in range(0, len(integerList)):
        integer_key = integerList[i]
        StatisticalMap[integer_key] = [0, 0]
    for index, row in df.iterrows():
        figure = int(row.委托价)
        amount = StatisticalMap[figure][0]
        volume = StatisticalMap[figure][1]
        StatisticalMap[figure] = [amount+row.数量, volume+row.总额]
    for key in integerList:
        result = result.append({'委托价': key, '数量':StatisticalMap[key][0], '总额':StatisticalMap[key][1]}, ignore_index = True)        
    return result
#print(getBriefOrderList(asks_df))


with pd.ExcelWriter(fileName) as writer:
    bids_df.to_excel(writer, sheet_name='委托买单(详细)')
    asks_df.to_excel(writer, sheet_name='委托卖单(详细)')
    getBriefOrderList(bids_df).to_excel(writer, sheet_name='委托买单(简略)')
    getBriefOrderList(asks_df).to_excel(writer, sheet_name='委托卖单(简略)')
print('success')

success
